In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
# Load the ideal weights data
weights_data = pd.read_csv("../../Dataset/Ideal_Votable_Supply_Data/Ideal_Weights.csv")

# Load the indexes data
data = pd.read_csv("../../Dataset/Ideal_Votable_Supply_Data/All_Parameters_Data.csv")

# Extract the weights with the highest total score
best_weights_row = weights_data.loc[weights_data['Total Score'].idxmax()]
weights_str = best_weights_row['Weights']

# Extract individual weights from the weights string (assuming 'w1', 'w2', 'w3', 'w4' format)
weights = {f"w{i+1}": float(val.split('=')[1]) for i, val in enumerate(weights_str.split(','))}

# Calculate the 'F' column using the extracted weights
data['F'] = round(
        (1 / (1 + (weights['w1'] * data['PR_Scaled']))) *
        (1 + (weights['w2'] * data['LAR_Scaled'])) * 
        (1 / (1 + (weights['w3'] * data['OP_Price_Scaled']))) *
        (1 + (weights['w4'] * data['AVPI_Scaled'])), 6)

# Calculate the 'Videal' column
f_max = data['F'].max() 
 
beta = 1.35
alpha = 0.7

F_min = data['F'].min()
F_max = data['F'].max()

data['IVS'] = round(data['Votable Supply'] * (beta + (np.exp(alpha * ((data['F'] - F_min) / (F_max - F_min))) - 1)), 2)

print('w1 -', weights['w1'])
print('w2 -', weights['w2'])
print('w3 -', weights['w3'])
print('w4 -', weights['w4'])

data.to_csv("../../Dataset/Ideal_Votable_Supply_Data/Calculated_IVS/All_Parameters_Data_with_IVS.csv", index=False)


In [ ]:
data = pd.read_csv("../../Dataset/Ideal_Votable_Supply_Data/Calculated_IVS/All_Parameters_Data_with_IVS.csv")

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
f_min = data['F'].min()
f_min

In [ ]:
f_max = data['F'].max()
f_max

In [ ]:
# Resample by month and take the mean
monthly_data = data.resample('M', on='Date')[['Votable Supply', 'IVS']].mean()

monthly_data['IVS/VS'] = round(monthly_data['IVS'] / monthly_data['Votable Supply'], 2)

# Save to CSV
monthly_data.to_csv('../../Dataset/Ideal_Votable_Supply_Data/Calculated_IVS/Calculated_Monthly_VS_and_IVS.csv', index=True)


In [ ]:
print(monthly_data['IVS/VS'].min())
print(monthly_data['IVS/VS'].max())

In [ ]:
monthly_data

In [ ]:
# Resample by quarter and take the mean
quarterly_data = data.resample('Q', on='Date')[['Votable Supply', 'IVS']].mean()

quarterly_data['IVS/VS'] = round(quarterly_data['IVS'] / quarterly_data['Votable Supply'], 2)

# Save to CSV
quarterly_data.to_csv('../../Dataset/Ideal_Votable_Supply_Data/Calculated_IVS/Calculated_Quarterly_VS_and_IVS.csv', index=True)


In [ ]:
quarterly_data

In [ ]:
# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 5))

# Plot the data
ax.plot(monthly_data.index, monthly_data["Votable Supply"], marker='o', color='blue', label="Votable Supply")
ax.plot(monthly_data.index, monthly_data["IVS"], marker='o', color='red', label="Ideal Votable Supply")

# Format y-axis in Millions
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.1f}M'))

# Labels and title
ax.set_xlabel("Date")
ax.set_ylabel("VS and IVS (in Millions)")
ax.set_title("VS and IVS Over Time")
ax.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Show the plot
plt.show()


In [ ]:
# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 5))

# Plot the data
ax.plot(data['Date'], data["Votable Supply"], color='blue', label="Votable Supply")
ax.plot(data['Date'], data["IVS"], color='red', label="Ideal Votable Supply")

# Format y-axis in Millions
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.1f}M'))

# Labels and title
ax.set_xlabel("Date")
ax.set_ylabel("VS and IVS (in Millions)")
ax.set_title("VS and IVS Over Time")
ax.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Show the plot
plt.show()


### Quarterly Results of IVS